# 서일님 코드 수정
* 코드 출처: https://colab.research.google.com/drive/1jvOR7fUyleb-Pd7i-GdNmCxOtkBZqx4b#scrollTo=0G2znWfJHxLI

In [ ]:
!pip install scikit-learn==0.23.2

In [ ]:
!pip install pycaret > /dev/null

# LGBM with pycaret



In [ ]:
import numpy as np
import pandas as pd
import os
import random
import pickle
from pycaret.classification import *
from pycaret.utils import check_metric
from datetime import timedelta, timezone, datetime

import time

# 데이터 가져오기

In [ ]:
#train_path = '/opt/ml/input/data/train_dataset/train_data.csv'
#test_path = '/opt/ml/input/data/train_dataset/test_data.csv'

# 도윤님 데이터 사용. 다른 데이터셋 사용 시 경로 수정할 것
train_path = "/opt/ml/input/data/train_data_time/train_all_three.csv"
test_path = "/opt/ml/input/data/train_data_time/test_all_three.csv"
submission_path = 'output/lgbm_results/'

In [ ]:
df_train_ori = pd.read_csv(train_path) 
df_test_ori = pd.read_csv(test_path)


answerCode2bool = {'userID':object,  'answerCode': 'int16', 'KnowledgeTag':object}
df_train_ori = df_train_ori.astype(answerCode2bool)
df_test_ori = df_test_ori.astype(answerCode2bool)
df_train_ori.Timestamp = pd.to_datetime(df_train_ori.Timestamp)
df_test_ori.Timestamp = pd.to_datetime(df_test_ori.Timestamp)

# Feature Engineering

## 미리미리 만들어놔야 하는 피처가 있습니다.

In [ ]:
# train에서 각 문제 평균 뽑기
testId_mean_sum = df_train_ori.groupby(['testId'])['answerCode'].agg(['mean','sum']).to_dict()
assessmentItemID_mean_sum = df_train_ori.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum']).to_dict()
KnowledgeTag_mean_sum = df_train_ori.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum']).to_dict()

assessmentItemID_mean_sum['mean']['A060001001']

In [ ]:
# 맞춰야하는 문항 ID 파악
set_assessmentItemID = set(df_test_ori.loc[df_test_ori.answerCode == -1, 'assessmentItemID'].values)

## 그때그때 만들어줘야 하는 피처가 있습니다.

In [ ]:
def feature_engineering(df):

    # 문항이 중간에 비어있는 경우를 파악 (1,2,3,,5)
    def assessmentItemID2item(x):
        return int(x[-3:]) - 1  # 0 부터 시작하도록 
    df['item'] = df.assessmentItemID.map(assessmentItemID2item)

    item_size = df[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()
    testId2maxlen = item_size.to_dict() # 중복해서 풀이할 놈들을 제거하기 위해

    item_max = df.groupby('testId').item.max()
    print(len(item_max[item_max + 1 != item_size]), '개의 시험지가 중간 문항이 빈다. item_order가 올바른 순서') # item_max는 0부터 시작하니까 + 1
    shit_index = item_max[item_max +1 != item_size].index
    shit_df = df.loc[df.testId.isin(shit_index),['assessmentItemID', 'testId']].drop_duplicates().sort_values('assessmentItemID')      
    shit_df_group = shit_df.groupby('testId')

    shitItemID2item = {}
    for key in shit_df_group.groups:
        for i, (k,_) in enumerate(shit_df_group.get_group(key).values):
            shitItemID2item[k] = i
        
    def assessmentItemID2item_order(x):
        if x in shitItemID2item:
            return int(shitItemID2item[x])
        return int(x[-3:]) - 1  # 0 부터 시작하도록 
    df['item_order'] =  df.assessmentItemID.map(assessmentItemID2item_order)



    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    # 유저가 푼 시험지에 대해, 유저의 전체 정답/풀이횟수/정답률 계산 (3번 풀었으면 3배)
    df_group = df.groupby(['userID','testId'])['answerCode']
    df['user_total_correct_cnt'] = df_group.transform(lambda x: x.cumsum().shift(1))
    df['user_total_ans_cnt'] = df_group.cumcount()
    df['user_total_acc'] = df['user_total_correct_cnt'] / df['user_total_ans_cnt']

    # 유저가 푼 시험지에 대해, 유저의 풀이 순서 계산 (시험지를 반복해서 풀었어도, 누적되지 않음)
    # 특정 시험지를 얼마나 반복하여 풀었는지 계산 ( 2번 풀었다면, retest == 1)
    df['test_size'] = df.testId.map(testId2maxlen)
    df['retest'] = df['user_total_ans_cnt'] // df['test_size']
    df['user_test_ans_cnt'] = df['user_total_ans_cnt'] % df['test_size']

    # 각 시험지 당 유저의 정확도를 계산
    df['user_test_correct_cnt'] = df.groupby(['userID','testId','retest'])['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_acc'] = df['user_test_correct_cnt']/df['user_test_ans_cnt']

    # 진홍님 feature
    df['assessment_category'] = df.apply(lambda row: row.assessmentItemID[2], axis = 1)#TODO # 대분류
    df['answer_mean'] = df.groupby('userID')['answerCode'].transform('mean') # 사용자별 정답률
    df['assessment_category_mean'] = df.groupby('assessment_category')['answerCode'].transform('mean') # 대분류의 정답률
    df['knowledge_tag_mean'] = df.groupby('KnowledgeTag')['answerCode'].transform('mean') # 지식 태그 분류별 정답률
    df['testId_answer_rate'] = df.groupby('testId')['answerCode'].transform('mean') #시험지 별로 정답률
    df['assessmentItemID_answer_rate'] = df.groupby('assessmentItemID')['answerCode'].transform('mean') #문항별 정답률
    '''
    def convert_time(s):
            timestamp = time.mktime(s.timetuple())
            return int(timestamp)

    df['Timestamp_int'] = df['Timestamp'].apply(convert_time)
    df['elapsed_time'] = df.loc[:,['userID','Timestamp_int','testId']].groupby(['userID','testId']).diff().shift(-1).fillna(int(10))
    
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    df['et_by_kt'] = df.groupby('KnowledgeTag')['elapsed_time'].transform(lambda x: x.quantile(q=0.5))#KT별 평균 소요 시간
    df['et_by_as'] = df.groupby('assessmentItemID')['elapsed_time'].transform(lambda x: x.quantile(q=0.5))#문항별 평균 소요 시간
    '''
    df['assessment_category2'] = df.apply(lambda row: row.assessmentItemID[-3:], axis=1) #문항의 대분류2
    df['answer_shift_-2'] = df.groupby('userID')['answerCode'].shift(-2).fillna(0)
    df['answer_shift_-1'] = df.groupby('userID')['answerCode'].shift(-1).fillna(0)

    # 과거 정보
    df['answer_shift_1'] = df.groupby('userID')['answerCode'].shift(1).fillna(0)
    df['answer_shift_2'] = df.groupby('userID')['answerCode'].shift(2).fillna(0)
    
    # 여기까지 진홍님
    
    # 도윤님 feature
    df['elapsed_time'] = df['elapsed'].apply(lambda x: np.log(x))
    
    # 아래의 피처는 다이나믹 합니다. 학습된 train의 평균값을 사용하지 않고, 새로 들어온 데이터의 평균 값을 사용합니다.
    # # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    # correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    # correct_t.columns = ["test_mean", 'test_sum']
    # correct_a = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum'])
    # correct_a.columns = ["ItemID_mean", 'ItemID_sum']
    # correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    # correct_k.columns = ["tag_mean", 'tag_sum']
    # df = pd.merge(df, correct_t, on=['testId'], how="left")
    # df = pd.merge(df, correct_a, on=['assessmentItemID'], how="left")
    # df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")

    # 본 피처는 train에서 얻어진 값을 그대로 유지합니다.
    df["test_mean"] = df.testId.map(testId_mean_sum['mean'])
    df['test_sum'] = df.testId.map(testId_mean_sum['sum'])
    df["ItemID_mean"] = df.assessmentItemID.map(assessmentItemID_mean_sum['mean'])
    df['ItemID_sum'] = df.assessmentItemID.map(assessmentItemID_mean_sum['sum'])
    df["tag_mean"] = df.KnowledgeTag.map(KnowledgeTag_mean_sum['mean'])
    df['tag_sum'] = df.KnowledgeTag.map(KnowledgeTag_mean_sum['sum'])

    return df

In [ ]:
df_test = feature_engineering(df_test_ori)
df = feature_engineering(df_train_ori)
# 피처를 대충만들어서 꽤 오래걸립니다.

In [ ]:
df_test_shift = df_test[df_test['userID'] != df_test['userID'].shift(-1)]

## 엔지니어링 이후의 EDA

In [ ]:
for i in range(14):
    order = df.loc[(df.item_order == df.user_test_ans_cnt) & (df.user_test_ans_cnt >= i),'answerCode'].mean()
    deorder = df.loc[(df.item_order != df.user_test_ans_cnt) & (df.user_test_ans_cnt >= i),'answerCode'].mean()
    print(order,deorder)

# 섞어서 풀면 인간의 성능이 항상 떨어진다고 생각했는데, 마지막 2놈은 아니었다.
# 그 와중에 뒤로 갈 수록 정답률 떨어지는건 신기하네

In [ ]:
# 시험지를 여러번 푼 경우
# 이 친구 보면 알겠지만, 나중에 푼다고 잘 푸는건 아님
# 어짜피 2번째 푸는거면 직전 문제들도 어련히 잘 풀었겠지
df[(df.userID == 1069)  & (df.testId =='A080000127')] 

# train/vali 데이터 셋 분리

In [ ]:
def my_train_vali_split(df, filter_option = None, train_must_exist_leaderboard = False, ratio=0.5, seed = 23):
    random.seed(seed)
    # 리더보드와 동일 조건의 컬럼 수집
    vali_full = df[(df['userID'] != df['userID'].shift(-1)) & (df.assessmentItemID.isin(set_assessmentItemID))].copy()
    
    # 리더보드와 동일 조건의 컬럼을 나누기
    ratio_r = (1 - ratio)
    vali_1 = vali_full.sample(frac=ratio_r, random_state = seed) # ratio가 1이면, ratio_r이 0이 되어, vali_1에 아무것도 할당되지 않는다.
    vali_2 = vali_full.drop(vali_1.index)

    # vali에 포함된 유저 목록 확인하기
    vali_1_userID = set(vali_1.userID.values)
    vali_2_userID = set(vali_2.userID.values)
    
    # vali에 없는 유저들만 train으로 데려오기
    train_1 = df[ df['userID'].isin(vali_1_userID) == False ].copy()
    train_2 = df[ df['userID'].isin(vali_2_userID) == False ].copy()
    
    # 마지막 응답만 가져올지 여부
    if filter_option == '시험지마지막응답':
        train_1 = train_1[train_1['testId'] != train_1['testId'].shift(-1)].copy()
        train_2 = train_2[train_2['testId'] != train_2['testId'].shift(-1)].copy()
    if filter_option == '사용자마지막응답':
        train_1 = train_1[train_1['userID'] != train_1['userID'].shift(-1)].copy()
        train_2 = train_2[train_2['userID'] != train_2['userID'].shift(-1)].copy()

    # train도 리더보드에서 맞춰야하는 문제(444개 문제)만 볼지 여부
    if train_must_exist_leaderboard:
        train_1 = train_1[train_1.assessmentItemID.isin(set_assessmentItemID)].copy()
        train_2 = train_2[train_2.assessmentItemID.isin(set_assessmentItemID)].copy()
    
    return train_1, vali_1, train_2, vali_2 , vali_full

## 후보기능

In [ ]:
def my_train_vali_split2(df, filter_option = None, train_must_exist_leaderboard = False, vali='Full', ratio=0.5, seed = 23):
    random.seed(seed)
    # 리더보드와 동일 조건의 컬럼 수집을 포기했다. df.assessmentItemID.isin(set_assessmentItemID))
    vali_full = df[(df['userID'] != df['userID'].shift(-1))].copy()

    # 리더보드와 동일 조건의 컬럼을 나누기
    ratio_r = (1 - ratio)
    vali_1 = vali_full.sample(frac=ratio_r, random_state = seed) # ratio가 1이면, ratio_r이 0이 되어, vali_1에 아무것도 할당되지 않는다.
    vali_2 = vali_full.drop(vali_1.index)

    # vali에 포함된 유저 목록 확인하기
    vali_1_userID = set(vali_1.userID.values)
    vali_2_userID = set(vali_2.userID.values)
    #vali_full_userID = set(vali_full.userID.values)

    # vali 유저의 전 기록을 쓸 경우, 디폴트는 마지막 응답만 사용합니다.
    if vali in ['Full', '시험지마지막응답']:
        vali_1 = df[ df['userID'].isin(vali_1_userID) == True ].copy()
        vali_2 = df[ df['userID'].isin(vali_2_userID) == True ].copy()
        #vali_full = df[ df['userID'].isin(vali_full_userID) == True ].copy()

    # vali 유저의 각 시험지 기록을 쓸 경우
    if vali == '시험지마지막응답': 
        vali_1 = vali_1[(vali_1['testId'] != vali_1['testId'].shift(-1))].copy()
        vali_2 = vali_2[(vali_2['testId'] != vali_2['testId'].shift(-1))].copy()
        #vali_full = df[ df['userID'].isin(vali_full_userID) == True ].copy()
        #vali_full = vali_full[(vali_full['testId'] != vali_full['testId'].shift(-1))].copy()

    # vali에 없는 유저들만 train으로 데려오기
    train_1 = df[ df['userID'].isin(vali_1_userID) == False ].copy()
    train_2 = df[ df['userID'].isin(vali_2_userID) == False ].copy()
    
    # 마지막 응답만 가져올지 여부
    if filter_option == '시험지마지막응답':
        train_1 = train_1[train_1['testId'] != train_1['testId'].shift(-1)].copy()
        train_2 = train_2[train_2['testId'] != train_2['testId'].shift(-1)].copy()
    if filter_option == '사용자마지막응답':
        train_1 = train_1[train_1['userID'] != train_1['userID'].shift(-1)].copy()
        train_2 = train_2[train_2['userID'] != train_2['userID'].shift(-1)].copy()

    # train도 리더보드에서 맞춰야하는 문제(444개 문제)만 볼지 여부
    if train_must_exist_leaderboard:
        train_1 = train_1[train_1.assessmentItemID.isin(set_assessmentItemID)].copy()
        train_2 = train_2[train_2.assessmentItemID.isin(set_assessmentItemID)].copy()
    
    return train_1, vali_1, train_2, vali_2 , vali_full

In [ ]:
def test_train_vali_split(df, filter_option = None, train_must_exist_leaderboard = False, vali='Full', ratio=0.5, seed = 23):
    random.seed(seed)
    # 리더보드와 동일 조건의 컬럼 수집을 포기했다.
    df = df[df.answerCode != -1].copy()
    vali_full = df[(df['userID'] != df['userID'].shift(-1))].copy()

    # 리더보드와 동일 조건의 컬럼을 나누기
    ratio_r = (1 - ratio)
    vali_1 = vali_full.sample(frac=ratio_r, random_state = seed) # ratio가 1이면, ratio_r이 0이 되어, vali_1에 아무것도 할당되지 않는다.
    vali_2 = vali_full.drop(vali_1.index)

    # vali에 포함된 유저 목록 확인하기
    vali_1_userID = set(vali_1.userID.values)
    vali_2_userID = set(vali_2.userID.values)
    #vali_full_userID = set(vali_full.userID.values)

    # vali 유저의 전 기록을 쓸 경우, 디폴트는 마지막 응답만 사용합니다.
    if vali in ['Full', '시험지마지막응답']:
        vali_1 = df[ df['userID'].isin(vali_1_userID) == True ].copy()
        vali_2 = df[ df['userID'].isin(vali_2_userID) == True ].copy()
        #vali_full = df[ df['userID'].isin(vali_full_userID) == True ].copy()

    # vali 유저의 각 시험지 마지막 기록을 쓸 경우
    if vali == '시험지마지막응답': 
        vali_1 = vali_1[(vali_1['testId'] != vali_1['testId'].shift(-1))].copy()
        vali_2 = vali_2[(vali_2['testId'] != vali_2['testId'].shift(-1))].copy()
        #vali_full = df[ df['userID'].isin(vali_full_userID) == True ].copy()
        #vali_full = vali_full[(vali_full['testId'] != vali_full['testId'].shift(-1))].copy()

    # vali에 없는 유저들만 train으로 데려오기
    train_1 = df[ df['userID'].isin(vali_1_userID) == False ].copy()
    train_2 = df[ df['userID'].isin(vali_2_userID) == False ].copy()
    
    # 마지막 응답만 가져올지 여부
    if filter_option == '시험지마지막응답':
        train_1 = train_1[train_1['testId'] != train_1['testId'].shift(-1)].copy()
        train_2 = train_2[train_2['testId'] != train_2['testId'].shift(-1)].copy()
    if filter_option == '사용자마지막응답':
        train_1 = train_1[train_1['userID'] != train_1['userID'].shift(-1)].copy()
        train_2 = train_2[train_2['userID'] != train_2['userID'].shift(-1)].copy()

    # train도 리더보드에서 맞춰야하는 문제(444개 문제)만 볼지 여부
    if train_must_exist_leaderboard:
        train_1 = train_1[train_1.assessmentItemID.isin(set_assessmentItemID)].copy()
        train_2 = train_2[train_2.assessmentItemID.isin(set_assessmentItemID)].copy()
    
    return train_1, vali_1, train_2, vali_2 , vali_full

# LGBM으로 실험 함수

In [ ]:
def exam_LGBM1( datasets ,FEATS, categorical_features=[],numeric_features=[],seed=47):
    train_1, vali_1, train_2, vali_2, vali_full = datasets
    random.seed(seed)
    settings = setup(data=train_1[FEATS], target='answerCode', train_size=0.8, categorical_features=categorical_features, numeric_features=numeric_features)
    
    lgbm = create_model('lightgbm', sort='AUC')
    tuned_lgbm = tune_model(lgbm, optimize = 'AUC', fold = 10)
    final_lgbm = finalize_model(tuned_lgbm)

    predict_model(lgbm)
    predict_model(tuned_lgbm)
    predict_model(final_lgbm)

    log = []
    prediction = predict_model(final_lgbm, data=vali_1[FEATS], raw_score = True)
    log.append(f"학습에 사용안한 데이터: {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    prediction = predict_model(final_lgbm, data=vali_2[FEATS], raw_score = True)
    log.append(f"학습에 사용한 데이터:  {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    prediction = predict_model(final_lgbm, data=vali_full[FEATS], raw_score = True)
    log.append(f"모든 vali 데이터:    {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
        
    return final_lgbm, log

In [ ]:
def exam_LGBM2( datasets ,FEATS, categorical_features=[],numeric_features=[],seed=47):
    train_1, vali_1, train_2, vali_2, vali_full = datasets
    random.seed(seed)
    settings = setup(data=train_2[FEATS], target='answerCode', train_size=0.8, categorical_features=categorical_features, numeric_features=numeric_features)
    
    lgbm = create_model('lightgbm', sort='AUC')
    tuned_lgbm = tune_model(lgbm, optimize = 'AUC', fold = 10)
    final_lgbm = finalize_model(tuned_lgbm)
    
    predict_model(lgbm)
    predict_model(tuned_lgbm)
    predict_model(final_lgbm)
    
    log = []
    prediction = predict_model(final_lgbm, data=vali_2[FEATS], raw_score = True)
    log.append(f"학습에 사용안한 데이터: {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    prediction = predict_model(final_lgbm, data=vali_1[FEATS], raw_score = True)
    log.append(f"학습에 사용한 데이터:  {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    prediction = predict_model(final_lgbm, data=vali_full[FEATS], raw_score = True)
    log.append(f"모든 vali 데이터:    {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")    
    
    return final_lgbm, log

In [ ]:
def exam_full(datasets, FEATS, categorical_features=[],numeric_features=[],seed=47):
    train_1, vali_1, train_2, vali_2, vali_full = datasets
    random.seed(seed)
    settings = setup(data=train_1[FEATS], target='answerCode', train_size=0.8, categorical_features=categorical_features, numeric_features=numeric_features)
    
    lgbm = create_model('lightgbm', sort='AUC')
    tuned_lgbm = tune_model(lgbm, optimize = 'AUC', fold = 10)
    final_lgbm = finalize_model(tuned_lgbm)

    predict_model(lgbm)
    predict_model(tuned_lgbm)
    predict_model(final_lgbm)

    log = []
    prediction = predict_model(final_lgbm, data=vali_full[FEATS], raw_score = True)
    log.append(f"모든 vali 데이터:    {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    return final_lgbm, log

# 절반 나눈 데이터로 실험

In [ ]:
df.columns # FEATS 참고용

In [ ]:
filter_option = '시험지마지막응답' # 시험지마지막응답, 사용자마지막응답, None
train_must_exist_leaderboard = False # True, False

FEATS = ['user_acc','ItemID_mean','answerCode']
FEATS += ['test_mean']

# 기본 columns 추가
FEATS += ['KnowledgeTag']

# 진홍님 features
FEATS += ['answer_shift_2']

categorical_features = []
#categorical_features = ['assessment_category2']
#categorical_features += ['KnowledgeTag']

numeric_features = []
numeric_features += []

In [ ]:
datasets = my_train_vali_split(df, filter_option = filter_option, train_must_exist_leaderboard = train_must_exist_leaderboard)
datasets_full = my_train_vali_split(df, filter_option = filter_option, train_must_exist_leaderboard = train_must_exist_leaderboard, ratio = 1.0) # 어떤 시드를 주던 영향 안받음.

print(len(datasets[0]), len(datasets[1]))
print(len(datasets[2]), len(datasets[3]))
print(len(datasets[4]), '\n')
print(datasets[0].answerCode.mean(),datasets[1].answerCode.mean())
print(datasets[2].answerCode.mean(),datasets[3].answerCode.mean())
print(datasets[4].answerCode.mean(), '\n')
print(len(datasets_full[0]), len(datasets_full[4]))
print(datasets_full[0].answerCode.mean(), datasets_full[4].answerCode.mean())

In [ ]:
# datasets = my_train_vali_split2(df, filter_option = filter_option, train_must_exist_leaderboard = train_must_exist_leaderboard, vali = '시험지마지막응답',  ratio = 0.5)
# datasets_full = my_train_vali_split2(df, filter_option = filter_option, train_must_exist_leaderboard = train_must_exist_leaderboard, vali = '시험지마지막응답', ratio =1.0) # 어떤 시드를 주던 영향 안받음.

# print(len(datasets[0]), len(datasets[1]))
# print(len(datasets[2]), len(datasets[3]))
# print(len(datasets[4]), '\n')
# print(datasets[0].answerCode.mean(),datasets[1].answerCode.mean())
# print(datasets[2].answerCode.mean(),datasets[3].answerCode.mean())
# print(datasets[4].answerCode.mean(), '\n')
# print(len(datasets_full[0]), len(datasets_full[4]))
# print(datasets_full[0].answerCode.mean(), datasets_full[4].answerCode.mean())

In [ ]:
# datasets = test_train_vali_split(df_test, filter_option = filter_option, train_must_exist_leaderboard = train_must_exist_leaderboard, vali = None,  ratio = 0.5)
# datasets_full = test_train_vali_split(df_test, filter_option = filter_option, train_must_exist_leaderboard = train_must_exist_leaderboard, vali = None, ratio =1.0) # 어떤 시드를 주던 영향 안받음.

# print(len(datasets[0]), len(datasets[1]))
# print(len(datasets[2]), len(datasets[3]))
# print(len(datasets[4]), '\n')
# print(datasets[0].answerCode.mean(),datasets[1].answerCode.mean())
# print(datasets[2].answerCode.mean(),datasets[3].answerCode.mean())
# print(datasets[4].answerCode.mean(), '\n')
# print(len(datasets_full[0]), len(datasets_full[4]))
# print(datasets_full[0].answerCode.mean(), datasets_full[4].answerCode.mean())

In [ ]:

model1,log1 = exam_LGBM1(datasets ,FEATS, categorical_features, numeric_features)
print('\n',model1)
print(f'\n폴드1:')
print('\n'.join(log1))


In [ ]:

model2,log2 = exam_LGBM2(datasets ,FEATS, categorical_features, numeric_features)
print(model2)
print(f'\n폴드2:')
print('\n'.join(log2))


In [ ]:
model_full,log_full = exam_full(datasets_full, FEATS, categorical_features, numeric_features)

print(model_full)
print(f'\n전부넣기:')
print('\n'.join(log_full))

# Inference

In [ ]:
# MAKE PREDICTION
# SAVE OUTPUT

prediction = predict_model(model_full, data=df_test_shift[FEATS], raw_score=True)
total_preds = prediction.Score_1.values

prediction_name = datetime.now(timezone(timedelta(hours=9))).strftime('%m%d_%H%M')

output_dir = 'output/lgbm-results/'
write_path = os.path.join(output_dir, f"{prediction_name}.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)    
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

name = ('filter_option', 'train_must_exist_leaderboard', 'FEATS', 'log1', 'log2','log_full')
logs = (filter_option, train_must_exist_leaderboard, FEATS, log1, log2,log_full)
#name = ('filter_option', 'train_must_exist_leaderboard', 'FEATS', 'log_full')
#logs = (filter_option, train_must_exist_leaderboard, FEATS, log_full)
write_path = os.path.join(output_dir, f"{prediction_name}_log.txt")
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    for n, l in zip(name, logs):
        w.write(f'{n}: {l}\n')
